In [3]:
import tweepy
import sys
import jupyter_black
import configparser

jupyter_black.load()


sys.path.append("./../app")
# import conn_utils
# from conn_utils import TwitterAPIConnector

# api_configs = TwitterAPIConnector(config_fpath="../app/config.ini")
# client_v1 = api_configs.get_twitter_conn_v1()
# client_v2 = api_configs.get_twitter_conn_v2()

# OPENAI_API_KEY = conn_utils.get_open_ai_key("../app/config.ini")

In [4]:
from tweets import Tweeter
from tweets import CreateTweet
import logging

module_logger = logging.getLogger(__name__)

t = Tweeter(config_fpath="../app/config.ini")
t.twitter_api_v2
# Uncomment to tweet
tweet_instance = CreateTweet()
#display(tweet_instance.quote)
#display(tweet_instance.work)
ai_tweet = tweet_instance.format_tweet()
# t.post_tweet("Hello World!")

while len(ai_tweet) > 280:
    print("*" * 10)
    print("Tweet too long!")
    module_logger.info("Tweet too long!")
    ai_tweet = tweet_instance.format_tweet()
print(f"Final tweet: \n {ai_tweet}")

module_logger.info(f"Final tweet: \n {ai_tweet}")
t.post_tweet(ai_tweet)

'Who is allowed to sin, sins less.'

'Amores (Love Affairs), Book II; xix, 34'

Final tweet: 
 "Who is allowed to sin, sins less." - Amores (Love Affairs), Book II; xix, 34
Freedom can reduce the allure of sin. Less restrictions, less rebellion.  📜

#OvidWisdom #OvidQuotes #quoteoftheday


In [5]:
print(ai_tweet)

"Happy is the man who has broken the chains which hurt the mind, and has given up worrying once and for all." from Metamorphoses
True happiness comes from freeing the mind of worries. It's about breaking mental chains.  📚

#Mindfulness #OvidQuotes #quoteoftheday


In [3]:
def get_utc_and_pacific_tm():
    from datetime import datetime
    import pytz

    # Get the current UTC time
    utc_now = datetime.utcnow()

    # Set the UTC timezone
    utc_timezone = pytz.timezone("UTC")

    # Convert UTC time to Pacific Standard Time (PST)
    pst_timezone = pytz.timezone("America/Los_Angeles")
    pst_time = utc_timezone.localize(utc_now).astimezone(pst_timezone)

    # Print the result
    print(f"UTC Time: {utc_now}")
    print(f"PST Time: {pst_time}")
    return pst_time, utc_now


get_utc_and_pacific_tm()

UTC Time: 2023-12-14 01:37:21.996205
PST Time: 2023-12-13 17:37:21.996205-08:00


(datetime.datetime(2023, 12, 13, 17, 37, 21, 996205, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>),
 datetime.datetime(2023, 12, 14, 1, 37, 21, 996205))

## Generate Tweets

**Steps:**
1) Sample quote 
2) Use TweetAI to generate commentary
3) Format quote and TweetAI content
4) Tweet/X it!

In [7]:
import pandas as pd
from genai import TweetAI
import re
import conn_utils

OPENAI_API_KEY = conn_utils.get_open_ai_key("../app/config.ini")

# Testing below
tweet_instance = CreateTweet()
display(tweet_instance.quote)
display(tweet_instance.work)
display(tweet_instance.format_tweet())

"Love yields to business. If you seek a way out of love, be busy; you'll be safe then."

'Remedia Amoris (The Cure for Love), lines 143–144'

'"Love yields to business. If you seek a way out of love, be busy; you\'ll be safe then." from Remedia Amoris (The Cure for Love), lines 143–144\nOvid\'s wisdom suggests that busyness can be an antidote to love\'s grip. Stay occupied, stay safe.  📜\n\n#LoveAdvice #OvidQuotes #quoteoftheday'

## Tweet Post Test

In [8]:
from pprint import pprint

tweet_instance = CreateTweet()
tweet_final = tweet_instance.format_tweet()
pprint(tweet_final)

while len(tweet_final) > 280:
    print("*" * 10)
    print("Tweet too long!")
    tweet_final = tweet_instance.format_tweet()


def post_tweet(tweet, api):
    tweet = api.create_tweet(text=tweet)


post_tweet(tweet=tweet_final, api=client_v2)

('"Who is allowed to sin, sins less." from Amores (Love Affairs), Book II; '
 'xix, 34\n'
 "When freedom to err is given, less mistakes are made. It's about trust and "
 'responsibility.  🏛️\n'
 '\n'
 '#OvidWisdom #OvidQuotes #quoteoftheday')
